<a href="https://colab.research.google.com/github/JianchaoSun/learning_machine_learning/blob/master/flower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive"

In [14]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from tensorflow.python.keras.layers.convolutional import Conv2D, MaxPooling2D 
tf.keras.__version__

'2.2.4-tf'

In [5]:
import os


import tarfile
tar = tarfile.open('drive/My Drive/flower/flower_photos.tgz', "r:gz")
tar.extractall('unzipped_folder')
tar.close()
base_path = 'unzipped_folder/flower_photos/'
categories = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

fnames = []
for category in categories:
    flower_folder = os.path.join(base_path, category)
    file_names = os.listdir(flower_folder)
    full_path = [os.path.join(flower_folder, file_name) for file_name in file_names]
    fnames.append(full_path)
print('length for each category:', [len(f) for f in fnames])

length for each category: [633, 898, 641, 699, 799]


In [0]:
import cv2
images = []
for names in fnames:
    one_category_images = [cv2.imread(name) for name in names if (cv2.imread(name)) is not None]
    images.append(one_category_images)

In [7]:
from skimage.transform import resize
img_width, img_height = 256, 256

img = images[3][659]
print(img.shape)
resized_img = resize(img, (img_width, img_height, 3))
resized_img2 = cv2.resize(img,(img_width, img_height), interpolation = cv2.INTER_CUBIC)

(213, 320, 3)


In [0]:
resized_images = []
for i,imgs in enumerate(images):
    resized_images.append([cv2.resize(img, (img_width, img_height), interpolation = cv2.INTER_CUBIC) for img in imgs])

In [0]:
from sklearn.model_selection import train_test_split
train_images = []
val_images = []
for imgs in resized_images:
    train, test = train_test_split(imgs, train_size=0.8, test_size=0.2)
    train_images.append(train)
    val_images.append(test)

In [10]:
from tensorflow.python.keras.utils import np_utils
import numpy as np

len_train_images = [len(imgs) for imgs in train_images]
print(len_train_images)
print('sum of train images:', np.sum(len_train_images))
train_categories = np.zeros((np.sum(len_train_images)), dtype='uint8')
for i in range(5):
    if i is 0:
        train_categories[:len_train_images[i]] = i
    else:
        train_categories[np.sum(len_train_images[:i]):np.sum(len_train_images[:i+1])] = i
        
len_val_images = [len(imgs) for imgs in val_images]
print(len_val_images)
print('sum of val_images:', np.sum(len_val_images))
val_categories = np.zeros((np.sum(len_val_images)), dtype='uint8')
for i in range(5):
    if i is 0:
        val_categories[:len_val_images[i]] = i
    else:
        val_categories[np.sum(len_val_images[:i]):np.sum(len_val_images[:i+1])] = i

[506, 718, 512, 559, 639]
sum of train images: 2934
[127, 180, 129, 140, 160]
sum of val_images: 736


In [0]:

tmp_train_imgs = []
tmp_val_imgs = []
for imgs in train_images:
    tmp_train_imgs += imgs
for imgs in val_images:
    tmp_val_imgs += imgs
train_images = np.array(tmp_train_imgs)
val_images = np.array(tmp_val_imgs)

In [12]:
print('Before converting')
print('train data:', train_images.shape)
print('train labels:', train_categories.shape)

train_data = train_images.astype('float32')
val_data = val_images.astype('float32')
train_labels = np_utils.to_categorical(train_categories, len(categories))
val_labels = np_utils.to_categorical(val_categories, len(categories))
print()
print('After converting')
print('train data:', train_data.shape)
print('train labels:', train_labels.shape)

Before converting
train data: (2934, 256, 256, 3)
train labels: (2934,)

After converting
train data: (2934, 256, 256, 3)
train labels: (2934, 5)


In [14]:
#build my own cnn
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from tensorflow.python.keras.layers.convolutional import Conv2D, MaxPooling2D 

model = tf.keras.Sequential()
model.add(Conv2D(32, (3,3), padding='same', input_shape = (256,256,3), activation='relu', name='conv_1'))
model.add(Conv2D(32, (3,3), activation='relu', name='conv_2'))
model.add(MaxPooling2D(pool_size=(2,2), name='maxpool_1'))
model.add(Dropout(0.45))

model.add(Conv2D(32, (3,3), padding='same', activation='relu', name='conv_3'))
model.add(Conv2D(32, (3,3), activation='relu', name='conv_4'))
model.add(MaxPooling2D(pool_size=(2,2), name='maxpool_2'))
model.add(Dropout(0.45))

model.add(Flatten())
model.add(Dense(512, activation='relu', name='dense_1'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu', name='dense_2'))
model.add(Dense(len(categories), name='output'))
model.add(Activation('softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
conv_2 (Conv2D)              (None, 254, 254, 32)      9248      
_________________________________________________________________
maxpool_1 (MaxPooling2D)     (None, 127, 127, 32)      0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 127, 127, 32)      0         
_________________________________________________________________
conv_3 (Conv2D)              (None, 127, 127, 32)      9248      
_________________________________________________________________
conv_4 (Conv2D)              (None, 125, 125, 32)      9248      
_________________________________________________________________
maxpool_2 (MaxPooling2D)     (None, 62, 62, 32)       

In [15]:
model.fit(train_data, train_labels, batch_size = 64, epochs = 10, validation_data = (val_data, val_labels))

Train on 2934 samples, validate on 736 samples
Epoch 1/10
2934/2934 [==============================] - 18s 6ms/sample - loss: 143.3718 - acc: 0.2219 - val_loss: 1.5945 - val_acc: 0.2283
Epoch 2/10
2934/2934 [==============================] - 13s 4ms/sample - loss: 1.5641 - acc: 0.2628 - val_loss: 1.5515 - val_acc: 0.3356
Epoch 3/10
2934/2934 [==============================] - 13s 4ms/sample - loss: 1.4378 - acc: 0.3715 - val_loss: 1.4048 - val_acc: 0.4389
Epoch 4/10
2934/2934 [==============================] - 13s 4ms/sample - loss: 1.2925 - acc: 0.4427 - val_loss: 1.3152 - val_acc: 0.4796
Epoch 5/10
2934/2934 [==============================] - 13s 4ms/sample - loss: 1.1594 - acc: 0.5276 - val_loss: 1.3035 - val_acc: 0.4579
Epoch 6/10
2934/2934 [==============================] - 13s 4ms/sample - loss: 1.0740 - acc: 0.5842 - val_loss: 1.2403 - val_acc: 0.5054
Epoch 7/10
2934/2934 [==============================] - 13s 5ms/sample - loss: 0.9428 - acc: 0.6339 - val_loss: 1.2286 - val_acc:

In [0]:
#use vgg19 pre trained model

from tensorflow.python.keras.applications import VGG19
model = VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
    
    # Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers[:1]:
  layer.trainable = False
  
    #Adding custom Layers 
x = model.output
x = tf.keras.layers.Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(len(categories), activation="softmax")(x)

# creating the final model 
vggmodel = tf.keras.models.Model(inputs = model.input, outputs = predictions)
vggmodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [17]:
vggmodel.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [21]:
vgg = vggmodel.fit(train_data, train_labels, batch_size = 64, epochs = 8, validation_data = (val_data, val_labels))

Train on 2934 samples, validate on 736 samples
Epoch 1/8
2934/2934 [==============================] - 64s 22ms/sample - loss: 8.7564 - acc: 0.2512 - val_loss: 1.5784 - val_acc: 0.3084
Epoch 2/8
2934/2934 [==============================] - 64s 22ms/sample - loss: 1.5509 - acc: 0.3132 - val_loss: 1.5025 - val_acc: 0.3288
Epoch 3/8
2934/2934 [==============================] - 64s 22ms/sample - loss: 1.4373 - acc: 0.3637 - val_loss: 1.3083 - val_acc: 0.4565
Epoch 4/8
2934/2934 [==============================] - 64s 22ms/sample - loss: 1.2919 - acc: 0.4414 - val_loss: 1.2222 - val_acc: 0.4823
Epoch 5/8
2934/2934 [==============================] - 65s 22ms/sample - loss: 1.2690 - acc: 0.4499 - val_loss: 1.1892 - val_acc: 0.5109
Epoch 6/8
2934/2934 [==============================] - 65s 22ms/sample - loss: 1.2577 - acc: 0.4530 - val_loss: 1.1841 - val_acc: 0.5177
Epoch 7/8
2934/2934 [==============================] - 65s 22ms/sample - loss: 1.2227 - acc: 0.4819 - val_loss: 1.1998 - val_acc: 0

In [26]:
#Task 4: Apply Data Augmentation to vgg 19
from tensorflow.python.keras.applications.vgg19 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

image_size = 224

datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   horizontal_flip=True,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2)

datagen.fit(train_data)


vggmodel.fit_generator(datagen.flow(train_data, train_labels, batch_size=32),
                    steps_per_epoch=len(train_data) / 32, epochs=10)



Epoch 1/10
92/91 [==============================] - 64s 697ms/step - loss: 1.3396 - acc: 0.4294
Epoch 2/10
92/91 [==============================] - 56s 612ms/step - loss: 1.2524 - acc: 0.4530
Epoch 3/10
92/91 [==============================] - 57s 620ms/step - loss: 1.2611 - acc: 0.4744
Epoch 4/10
92/91 [==============================] - 57s 625ms/step - loss: 1.1765 - acc: 0.5027
Epoch 5/10
92/91 [==============================] - 58s 628ms/step - loss: 1.1683 - acc: 0.5109
Epoch 6/10
92/91 [==============================] - 58s 630ms/step - loss: 1.1579 - acc: 0.5194
Epoch 7/10
92/91 [==============================] - 58s 631ms/step - loss: 1.1380 - acc: 0.5382
Epoch 8/10
92/91 [==============================] - 58s 631ms/step - loss: 1.1452 - acc: 0.5392
Epoch 9/10
92/91 [==============================] - 58s 632ms/step - loss: 1.1176 - acc: 0.5481
Epoch 10/10
92/91 [==============================] - 58s 632ms/step - loss: 1.0773 - acc: 0.5596
